In [3]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from prepare import basic_clean, tokenize, lemmatize, stem, remove_stopwords, prep_repo_data, split_data
import re

In [4]:
def get_repo_names():
    repo_names = []

    for page in range(1,10):
        print(f'Scraping page {page}...')
        link = f'https://github.com/search?p={page}&q=zombie&type=Repositories'
        print(f'Address: {link}')
        response = requests.get(link, headers={'user-agent': 'Codeup DS Hopper'})
        soup = BeautifulSoup(response.text)
        print(f"First repository on page... {soup.find_all('a',class_='v-align-middle')[0]['href']}")
        time.sleep(5)
        for repository in range(0,len(soup.find_all('a',class_='v-align-middle'))):
            repo_names.append(soup.find_all('a', class_='v-align-middle')[repository]['href'])
    time.sleep(10)

    for page in range(11,20):
        print(f'Scraping page {page}...')
        link = f'https://github.com/search?p={page}&q=zombie&type=Repositories'
        print(f'Address: {link}')
        response = requests.get(link, headers={'user-agent': 'Codeup DS Hopper'})
        soup = BeautifulSoup(response.text)
        print(f"First repository on page... {soup.find_all('a',class_='v-align-middle')[0]['href']}")
        time.sleep(5)
        for repository in range(0,len(soup.find_all('a',class_='v-align-middle'))):
            repo_names.append(soup.find_all('a', class_='v-align-middle')[repository]['href'])
    return repo_names

In [5]:
# repo_names = get_repo_names()
# repo_names

In [6]:
df = pd.read_json('data.json')
df.head()

,repo,language,readme_contents
0,assaf/zombie,JavaScript,"# Zombie.js\n### Insanely fast, headless full-..."
1,IAIK/ZombieLoad,C,# ZombieLoad PoC\n\nThis repository contains s...
2,marblexu/PythonPlantsVsZombies,Python,# Python版植物大战僵尸\n 一个简单的植物大战僵尸游戏。\n \n `仅供个人...
3,amazon-archives/aws-lambda-zombie-workshop,JavaScript,# This workshop is no longer actively being ma...
4,ErLinErYi/PlantsVsZombies,C++,# PlantsVsZombies\n[[English]](README_en_us.md...


In [7]:
# df['clean'] = [tokenize(basic_clean(readme)) for readme in df.readme_contents]

In [8]:
# df['clean'] = [re.sub('[\n]','', readme) for readme in df.clean]

In [9]:
# df['stemmed'] = [remove_stopwords(stem(readme)) for readme in df.clean]

In [10]:
# df['lemmatized'] = [remove_stopwords(lemmatize(readme)) for readme in df.clean]

In [11]:
df = prep_repo_data(df)

/Users/joshua/codeup-data-science/nlp-project/prepare.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
/Users/joshua/codeup-data-science/nlp-project/prepare.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [18]:
df.language.value_counts()

JavaScript    23
C#            22
SourcePawn    20
PHP           20
HTML          20
C             20
C++           16
Java          15
Lua           11
Python        11
Ruby          10
Name: language, dtype: int64

In [13]:
# languages = df.language.value_counts()[df.language.value_counts() >= 11].index.to_list()
# df = df[df.language.isin(languages)]

In [14]:
# df.drop_duplicates(inplace=True)

In [15]:
# df.language.value_counts()

In [16]:
# languages = ['HTML','SourcePawn','C','PHP','Ruby']
# repo_names = []

# for page in range(1,10):
#     for language in languages:
#         print(f'Scraping page {page} of {language}...')
#         link = f'https://github.com/search?l={language}&p={page}&q=zombie&type=Repositories'
#         print(f'Address: {link}')
#         response = requests.get(link, headers={'user-agent': 'Codeup DS Hopper'})
#         soup = BeautifulSoup(response.text)
#         print(f"First repository on page... {soup.find_all('a',class_='v-align-middle')[0]['href']}")
#         time.sleep(5)
#         for repository in range(0,len(soup.find_all('a',class_='v-align-middle'))):
#             repo_names.append(soup.find_all('a', class_='v-align-middle')[repository]['href'])
# time.sleep(10)


# for page in range(11,20):
#     for language in languages:
#         print(f'Scraping page {page} of {language}...')
#         link = f'https://github.com/search?l={language}&p={page}&q=zombie&type=Repositories'
#         print(f'Address: {link}')
#         response = requests.get(link, headers={'user-agent': 'Codeup DS Hopper'})
#         soup = BeautifulSoup(response.text)
#         print(f"First repository on page... {soup.find_all('a',class_='v-align-middle')[0]['href']}")
#         time.sleep(5)
#         for repository in range(0,len(soup.find_all('a',class_='v-align-middle'))):
#             repo_names.append(soup.find_all('a', class_='v-align-middle')[repository]['href'])

In [17]:
# repo_names

In [19]:
train, validate, test = split_data(df)

ValueError: too many values to unpack (expected 2)